In [ ]:
%matplotlib inline  
import os
import re
import numpy as np
import pandas as pd
import pickle
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import json
from result_parser import get_exp_result
from utils.distribution import distr_profile


#fig_width = 10
#plt.rcParams["figure.figsize"] = (fig_width, 6)
#plt.rcParams['font.size'] = 22


In [ ]:
data_distr = 'gbalan' # htail, cus
VI = 3
epoch = 500
fig_zoom = 0.3

# compare over datasets
acc_threshold = 95
metrics = {'ACC':[], 'ETA':[], 'TTA':[]}
x_labels = []
y_labels = ['Final Accuracy (%)',
            'Epoch to Accuracy',
            'Time to Accuracy (sec)']
y_lim = [80, epoch, 4000]
'''
y_labels = ['Final Accuracy(%)',
            'Epoch to Reach {}% Best Acc'.format(acc_threshold),
            'Time to Reach {}% Best Acc'.format(acc_threshold)]
'''
legends = {'fedavg':'FedAvg', 'oort_e0.8':'Oort', 'algo3_deg1_e0.8_wof100':'Fedcism'}
textures = ['xx', '//', None]

for dataset in ['cifar10']:
    for shard_per_user in [10, 2]:
        for frac in [0.1, 0.05]:
            x_labels.append('$S={}$\n$C={}$'.format(shard_per_user, frac))
            bacc_stack = []
            TTAe_stack = []
            TTAt_stack = []
            
            for RUN in range(3):
        
                data_distr_file = '{}_dict_users_vi{}_r{}.pkl'.format(data_distr, VI, RUN)
                result_folder = 'cossim_{}_vi{}_r{}'.format(data_distr, VI, RUN)
                '''if frac == 0.1:
                    dpath = '../save/{}/data_distr/num100/shard{}/{}'.format(dataset, shard_per_user, data_distr_file)
                    profile = distr_profile(dpath)
                    _ =profile.plot_local_distr(shard_per_user)
    '''
                algos, color_algos, bacc_algos, required_time, _ = get_exp_result(
                    dataset, data_distr, VI, RUN, data_distr_file, result_folder, legends,
                    shard_per_user, frac, acc_threshold, show_fig=True, save=True)

                bacc_stack.append(bacc_algos)
                TTAe_stack.append(required_time['epoch'])
                TTAt_stack.append(required_time['time_simu'])
            
            metrics['ACC'].append(bacc_stack)
            metrics['ETA'].append(TTAe_stack)
            metrics['TTA'].append(TTAt_stack)
    
    # summary 
    x = np.arange(len(x_labels))
    for i, (k, v) in enumerate(metrics.items()):
        fig, ax = plt.subplots(figsize=(16*fig_zoom, 9*fig_zoom))
        #plt.title('{} - {} - vi{}'.format(dataset, data_distr, VI), y=1.1)
        if data_distr == 'cus': # alias
            fig_name = '{}_{}_vi{}'.format(k, 'sparsez', VI)
        else:            
            fig_name = '{}_{}_vi{}'.format(k, data_distr, VI)
        fig_path = os.path.join('..', 'save', dataset, 'fig', fig_name)
        
        cur_algos = algos
        cur_v_T = v_T = np.array(v).T
        if 'TA' in k:
            cur_algos = algos[1:]
            cur_v_T = v_T[1:] # [algo][run][data_config]
        for aidx, algo in enumerate(cur_algos):
            w = 0.9/len(cur_algos)
            y_avg = cur_v_T[aidx].mean(axis=0)
            y_max = cur_v_T[aidx].max(axis=0)
            y_min = cur_v_T[aidx].min(axis=0)
            y_err = np.vstack((y_max-y_avg, y_avg-y_min))
            
            if k == 'ACC':
                p = ax.bar(x+aidx*w-0.45+w/2, y_avg, yerr=y_err, 
                            label=legends[algo], width=w,
                            capsize=4, hatch=textures[aidx])
            else:
                p = ax.bar(x+aidx*w-0.45+w/2, y_avg, yerr=y_err, 
                            label=legends[algo], width=w, color=color_algos[aidx+1],
                            capsize=4, hatch=textures[aidx+1])
                
                if aidx == 0:
                    base_yavg = y_avg
                elif aidx == 1:
                    txt = ['{:.2f}x'.format(x) for x in base_yavg/y_avg]
                    ax.bar_label(p, labels=txt, padding=3)

        ax.set_ylabel(y_labels[i])
        ax.set_xlabel('Data Configuration')
        ax.set_ylim(top=y_lim[i])
        ax.set_xticks(x, labels=x_labels)#, rotation='vertical')
        #fig.legend(fancybox=True, shadow=True, fontsize=16,
        #                      loc='lower left', bbox_to_anchor=(1, 0))
        ax.legend(fontsize=8, ncol=len(cur_algos), frameon=False)

        # Hide the right and top spines
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
    
        fig.tight_layout()
        fig.savefig('{}.{}'.format(fig_path, 'png'), format='png', transparent=True)
        fig.savefig('{}.{}'.format(fig_path, 'eps'), format='eps', transparent=True)